In [ ]:
# imports
import os
from types import SimpleNamespace

import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import zscore
import pandas as pd
import csv
import matplotlib.colors as mcolors


import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

In [ ]:
metrics={}

### Load track2p folder and suite2p datasets

In [ ]:
#load all the datasets
def find_t2p_directories(parent_directory):
    # List all items in the parent directory
    items = os.listdir(parent_directory)
    # Filter the list to include only directories that start with "t2p"
    t2p_directories = [item for item in items if item.startswith('t2p') and os.path.isdir(os.path.join(parent_directory, item))]
    return t2p_directories

parent_directory = '/Users/manonmantez/Desktop/jm038/' 
t2p_dirs = find_t2p_directories(parent_directory)
print(t2p_dirs)

In [ ]:
path = 't2p_rigid_chan1_all_'
print(path)
plane = 'plane0' # which plane to process (the example dataset is single-plane)
option= 1 #1: Registration done of all days, 2 : Registration done of two days , 3 : Registration done of three days
print(option)
days = ['P8', 'P9', 'P10' ,'P11', 'P12','P13','P14']

In [ ]:
if option == 2 :
    second_day= 'P9' # change this !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    if second_day in days:
        index_of_second_day = days.index(second_day)
        print(index_of_second_day)

In [ ]:
if option == 3 :
    second_day= 'P13' # change this
    third_day= 'P14' # change this
    if third_day in days:
        index_of_second_day = days.index(second_day)
        index_of_third_day = days.index(third_day)
        print(index_of_second_day)
        print(index_of_third_day)

In [ ]:
t2p_save_path = os.path.join(parent_directory, path)
print(t2p_save_path)

In [ ]:
# np.load() the match matrix (plane0_match_mat.npy)
t2p_match_mat = np.load(os.path.join(t2p_save_path,  'track2p',f'{plane}_match_mat.npy'), allow_pickle=True)
# np.load() settings (this contains suite2p paths etc.) (track_ops.npy)
track_ops_dict = np.load(os.path.join(t2p_save_path, 'track2p', 'track_ops.npy'), allow_pickle=True).item()
track_ops = SimpleNamespace(**track_ops_dict) # create dummy object from the track_ops dictionary
print(track_ops.reg_chan)
print(track_ops.transform_type)

In [ ]:
print('Datasets used for t2p:\n')
for ds_path in track_ops.all_ds_path:
    print(ds_path)

print('\nMatch matrix shape:')
print(t2p_match_mat.shape)

## Above is the output of the algorithm, now generate ground truth
### Generate grid

In [ ]:
# load the suite2p output of the first dataset
s2p_path = os.path.join(track_ops.all_ds_path[0], 'suite2p', plane)

# load the ops and stat
ops_first = np.load(os.path.join(s2p_path, 'ops.npy'), allow_pickle=True).item()
stat_first = np.load(os.path.join(s2p_path, 'stat.npy'), allow_pickle=True)
iscell_first = np.load(os.path.join(s2p_path, 'iscell.npy'), allow_pickle=True)
s2p_inds = np.arange(len(stat_first))

# filter stat based on the track2p probability
print(f'Filtering stat based on the track2p probability (iscell_thr={track_ops.iscell_thr})')
s2p_inds_iscell = s2p_inds[iscell_first[:, 0] > track_ops.iscell_thr]
stat_first_iscell = stat_first[iscell_first[:, 0] > track_ops.iscell_thr]
print(f'Filtered stat from {len(stat_first)} to {len(stat_first_iscell)} cells')



In [ ]:
# now plot the fov with contours
plt.figure(figsize=(10, 10))
plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(len(stat_first_iscell)):
    # get random color 
    color = np.random.rand(3)
    plt.scatter(stat_first_iscell[i]['xpix'], stat_first_iscell[i]['ypix'], s=1, color=color, alpha=0.5)

In [ ]:
# now make a 8x8 grid on the image
n_grid = 9
grid_x = np.arange(0, ops_first['meanImg'].shape[1], ops_first['meanImg'].shape[1] // n_grid)
grid_y = np.arange(0, ops_first['meanImg'].shape[0], ops_first['meanImg'].shape[0] // n_grid)

plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(n_grid):
    plt.axvline(grid_x[i], color='r')
    plt.axhline(grid_y[i], color='r')

plt.axis('off')

In [ ]:
# now for each grid cross get the nearest cell (except for 0 and 512 pixels)
all_cell_med = []

# get all cell medians from stat
for i in range(len(stat_first_iscell)):
    all_cell_med.append(stat_first_iscell[i]['med'])
    

In [ ]:
# compute coordinates of the grid crosses
grid_crosses = []
for i in range(1, n_grid):
    for j in range(1, n_grid):
        grid_crosses.append((grid_x[i], grid_y[j]))

grid_crosses

In [ ]:
plt.imshow(ops_first['meanImg'], cmap='gray')
for i in range(n_grid):
    plt.axvline(grid_x[i], color='gray')
    plt.axhline(grid_y[i], color='gray')
for cross in grid_crosses:
    plt.scatter(cross[0], cross[1], s=20, color='r')

plt.axis('off')


In [ ]:
# compute distances between each cell and each grid cross
distances = np.zeros((len(stat_first_iscell), len(grid_crosses)))
for i, cell_med in enumerate(all_cell_med):
    for j, cross in enumerate(grid_crosses):
        distances[i, j] = np.sqrt((cell_med[0] - cross[0])**2 + (cell_med[1] - cross[1])**2)

In [ ]:
# now get the nearest cell for each grid cross
nearest_cells = np.argmin(distances, axis=0)
nearest_cells

In [ ]:
plt.imshow(ops_first['meanImg'], cmap='gray')

for cross in grid_crosses:
    plt.scatter(cross[0], cross[1], s=20, color='C0')

plt.axis('off')

for i, cell in enumerate(nearest_cells):
    roi = np.zeros_like(ops_first['meanImg'])
    roi[stat_first_iscell[cell]['ypix'], stat_first_iscell[cell]['xpix']] = 1
    plt.contour(roi, levels=[0.5], colors='C1')


In [ ]:
# now get the original indices
original_indices = s2p_inds_iscell[nearest_cells]
print('Now open suite2p and try to manually track cells:')
[print(f'{original_indices[i]}') for i in range(len(original_indices))]

In [ ]:
# make the same plot as above but with numbers labeled next to the roi
plt.figure(figsize=(10, 10))
# clip based on 99.99 percentile
img = ops_first['meanImg'].copy()
img[img > np.percentile(img, 99.99)] = np.percentile(img, 99.99)
plt.imshow(img, cmap='gray')

for i, cell in enumerate(nearest_cells):
    roi = np.zeros_like(ops_first['meanImg'])
    roi[stat_first_iscell[cell]['ypix'], stat_first_iscell[cell]['xpix']] = 1
    plt.contour(roi, levels=[0.5], colors='C1')
    plt.text(stat_first_iscell[cell]['med'][1]+10, stat_first_iscell[cell]['med'][0]-10, f'{original_indices[i]}', color='C1')
    plt.scatter(grid_crosses[i][0], grid_crosses[i][1], s=20, color='C0', marker='x')

plt.axis('off')

### Getting real suite2p indexes to compare it to

In [ ]:
def function(path, plane):

    track_ops_dict = np.load(os.path.join(path, "track2p", "track_ops.npy"), allow_pickle=True).item()
    track_ops = SimpleNamespace(**track_ops_dict)
    t2p_match_mat = np.load(os.path.join(path,"track2p" ,f"{plane}_match_mat.npy"), allow_pickle=True)
    t2p_match_mat_allday = t2p_match_mat
    
    if track_ops.iscell_thr is not None:
        true_indices= np.empty((len(t2p_match_mat), len(track_ops.all_ds_path)), dtype=object)
        
    for j in range(len(t2p_match_mat)):
        for (i, ds_path) in enumerate(track_ops.all_ds_path):
            iscell = np.load(os.path.join(ds_path, 'suite2p', plane, 'iscell.npy'), allow_pickle=True)
            if track_ops.iscell_thr is not None:
                s2p_indexes= np.where(iscell[:,1]>track_ops.iscell_thr)[0]
                t2p_index=t2p_match_mat[j,i]

                if t2p_index is None:
                    true_index = None
                else:
                    true_index=s2p_indexes[t2p_index]

                true_indices[j, i] = true_index
            else:
                pass
    if track_ops.iscell_thr is not None:        
        np.save(os.path.join(path, "true_indices.npy"), true_indices)
        print("saved")
    else:
        print("not in locals")
        
function(t2p_save_path, plane)  

indices_npy= np.load(os.path.join(t2p_save_path,"true_indices.npy"), allow_pickle=True)
print(indices_npy)
print(indices_npy.shape)

In [ ]:
np.sort(original_indices)

In [ ]:
# index rows for cells where the first row is in original_indices
all_s2p_idxs = []
for idx in np.sort(original_indices):
    idx_row = np.where(indices_npy[:,0]==idx)

    if len(idx_row[0]) == 0:
        s2p_idxs = [None]*len(track_ops.all_ds_path)
        #print('here')

    else:
        s2p_idxs = indices_npy[idx_row]
    all_s2p_idxs.append(s2p_idxs[0])

all_s2p_idxs = np.array(all_s2p_idxs)

#print(all_s2p_idxs.dtype)
print(all_s2p_idxs)

### Load 'ground truth' (csv table)

In [ ]:
manual_matches = np.genfromtxt(os.path.join(parent_directory, 'manual_matches.csv'), delimiter=';', skip_header=1 )
manual_matches = np.where(np.isnan(manual_matches), None, manual_matches)
manual_matches = [[int(x) if x is not None else None for x in row] for row in manual_matches]
manual_matches

In [ ]:
for i in range(len(manual_matches)):
    # Vérifier si tous les éléments sauf le premier sont "None"
    if all(x == None for x in manual_matches[i][1:]):
        manual_matches[i] = None

In [ ]:
for i in range(len(manual_matches)):
    # print the rows
    print(f'track2p:     {all_s2p_idxs[i]}')
    print(f'manual:      {manual_matches[i]}')
    print('')

In [ ]:
def compute_metrics_all_days(all_s2p_idxs, manual_matches):

    all_s2p_idxs_all_days = all_s2p_idxs.copy()
    manual_matches_all_days = manual_matches.copy()

    #IoU=1
    for i in range(len(manual_matches)):
        if all_s2p_idxs_all_days[i] is not None:
            if any(x is None for x in all_s2p_idxs_all_days[i]):
                all_s2p_idxs_all_days[i] = None
        if manual_matches_all_days[i] is not None:
            if any(x is None for x in manual_matches_all_days[i]):
                manual_matches_all_days[i] = None


       #print(f'track2p:     {all_s2p_idxs[i]}')
       # print(f'manual:      {manual_matches[i]}')
       #print('')
        print(f'track2p:     {all_s2p_idxs_all_days[i]}')
        print(f'manual:      {manual_matches_all_days[i]}')
       # print('')


    count_FP=0
    count_TP=0
    count_FN=0
    count_IS=0
    count_TN=0

    nb_days= len(track_ops.all_ds_path)

    for i in range(len(manual_matches_all_days)):
        if all_s2p_idxs_all_days[i] is None and manual_matches_all_days[i] is None:
             count_TN +=1
        # if track2p detected on all days 
        elif all_s2p_idxs_all_days[i] is None and manual_matches_all_days[i] is not None:
                count_FN +=1
                print('FN')
                print( all_s2p_idxs_all_days[i] )
                print( manual_matches_all_days[i] )
        elif all_s2p_idxs_all_days[i] is not None and manual_matches_all_days[i] is None:
                count_FP +=1
                print('FP')
                print( all_s2p_idxs_all_days[i] )
                print( manual_matches_all_days[i] )
        elif all_s2p_idxs_all_days[i] is not None and manual_matches_all_days[i] is not None:
            if any((x!=y) for x, y in zip(all_s2p_idxs_all_days[i], manual_matches_all_days[i])):
                    count_IS +=1
                    print('FP')
                    print( all_s2p_idxs_all_days[i] )
                    print( manual_matches_all_days[i] )
            else:
                    count_TP +=1

    print(f'FP: {count_FP}')
    print(f'TP: {count_TP}')
    print(f'FN: {count_FN}')
    print(f'IS: {count_IS}')

    TP_rate= count_TP/len(manual_matches_all_days)
    TN_rate= count_TN/len(manual_matches_all_days)
    FP_rate= count_FP/len(manual_matches_all_days)
    FN_rate= count_FN/len(manual_matches_all_days)
    IS_rate= count_IS/len(manual_matches_all_days)


    total_rate= TP_rate + TN_rate + FP_rate + FN_rate + IS_rate
    print(f'Sum of rates: {total_rate:.2f}')

    precision = count_TP/(count_TP+count_FP+count_IS)
    recall = count_TP/(count_TP+count_FN)
    f1 = 2*(precision*recall)/(precision+recall)
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1: {f1}')


    filename= os.path.join(t2p_save_path,'metrics_tracking_all_days.csv')
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file ,delimiter=";")
        writer.writerow(['TP', TP_rate])
        writer.writerow(['TN', TN_rate])
        writer.writerow(['FP', FP_rate])
        writer.writerow(['FN', FN_rate])
        writer.writerow(['IS', IS_rate])
        writer.writerow(['Precision', round(precision,2)])
        writer.writerow(['Recall', round(recall,2)])
        writer.writerow(['F1', round(f1,2)])
    print(f"CSV file '{filename}' created successfully.")
        

In [ ]:
compute_metrics_all_days(all_s2p_idxs, manual_matches)

### functions to get metrics (FP,FN,F1...)

In [ ]:
def calculate_metrics_pairs(manual_matches_elements,all_s2p_idxs_elements):
    
    TP_indexes=[]
    FP_indexes=[]
    TN_indexes=[]
    FN_indexes=[]
    IS_indexes=[]

    for i in range(len(manual_matches_elements)):

        if  all_s2p_idxs_elements[i][1] == manual_matches_elements[i][1] and all_s2p_idxs_elements[i][1] is not None:
            TP_indexes.append(i)

        elif all_s2p_idxs_elements[i][1] == manual_matches_elements[i][1] and all_s2p_idxs_elements[i][1] is None: 
            TN_indexes.append(i)
    
        elif manual_matches_elements[i][1] is None and all_s2p_idxs_elements[i][1] is not None:
            FP_indexes.append(i)
    
        elif all_s2p_idxs_elements[i][1] is None and manual_matches_elements[i][1] is not None:
            FN_indexes.append(i)

        elif all_s2p_idxs_elements[i][1] != manual_matches_elements[i][1] :
            IS_indexes.append(i)
            #FP_indexes.append(i)

    TP_rate= len(TP_indexes)/len(manual_matches_elements)
    TN_rate= len(TN_indexes)/len(manual_matches_elements)
    FP_rate= len(FP_indexes)/len(manual_matches_elements)
    FN_rate= len(FN_indexes)/len(manual_matches_elements)
    IS_rate= len(IS_indexes)/len(manual_matches_elements)


    print(f'TP: {len(TP_indexes)}   {TP_rate:.2f} \n')
    print(f'TN: {len(TN_indexes)}   {TN_rate:.2f} \n')
    print(f'FP: {len(FP_indexes)}   {FP_rate:.2f} \n')
    print(f'FN: {len(FN_indexes)}   {FN_rate:.2f} \n')
    print(f'IS: {len(IS_indexes)}   {IS_rate:.2f} \n')

    correctly_assign_link_rate = TP_rate + TN_rate
    incorrect_assign_link_rate = FP_rate + FN_rate + IS_rate

    print(f'correctly_assign_link_rate: {correctly_assign_link_rate:.2f}')
    print(f'incorrect_assign_link_rate: {incorrect_assign_link_rate:.2f}')

    total_rate= TP_rate + TN_rate + FP_rate + FN_rate + IS_rate
    print(f'Sum of rates: {total_rate:.2f}')

    precision= len(TP_indexes)/(len(TP_indexes) + (len(FP_indexes)+ len(IS_indexes)))
    recall= len(TP_indexes)/(len(TP_indexes) + len(FN_indexes))
    f1_score= (2*(precision*recall))/(precision+recall)

    return TP_rate, TN_rate, FP_rate, FN_rate, IS_rate, correctly_assign_link_rate, incorrect_assign_link_rate, precision, recall, f1_score



In [ ]:
def write_metrics_to_csv(TP_rate_all_days, TN_rate_all_days, FP_rate_all_days, FN_rate_all_days, IS_rate_all_days,correctly_assign_link_rate_all_days, incorrect_assign_link_rate_all_days, precision_all_days, recall_all_days, f1_score_all_days, option):
        
    if option ==1 :
        row_names = ['TP_rate', 'TN_rate', 'FP_rate', 'FN_rate', 'IS_rate', 'correctly_assign_link_rate', 'incorrect_assign_link_rate', 'precision', 'recall', 'f1_score']
        column_names = ['P9', 'P10', 'P11', 'P12', 'P13', 'P14']

                        # Créer une liste de listes pour les données
        data = [
                TP_rate_all_days,
                TN_rate_all_days,
                FP_rate_all_days,
                FN_rate_all_days,
                IS_rate_all_days,
                correctly_assign_link_rate_all_days,
                incorrect_assign_link_rate_all_days,
                precision_all_days,
                recall_all_days,
                f1_score_all_days
                ]
            

            # Ajouter les noms des lignes
        data_with_row_names = [[row_names[i]] + [round(value, 2) for value in data[i]] for i in range(len(row_names))]

            
        filename= os.path.join(t2p_save_path,'metrics.csv')
            # Écrire les données dans un fichier CSV
        with open(filename, 'w', newline='') as file:
                writer = csv.writer(file, delimiter=';')
                writer.writerow([''] + column_names)  # Écrire les noms des colonnes
                writer.writerows(data_with_row_names)  # Écrire les données avec les noms des lignes

            
        print(f"CSV file '{filename}' created successfully.")
    
    if option ==2: 
                
        row_names = ['TP_rate', 'TN_rate', 'FP_rate', 'FN_rate', 'IS_rate', 'correctly_assign_link_rate', 'incorrect_assign_link_rate', 'precision', 'recall', 'f1_score']
        column_names = [days[index_of_second_day]]

                        # Créer une liste de listes pour les données
        data = [
                TP_rate_all_days,
                TN_rate_all_days,
                FP_rate_all_days,
                FN_rate_all_days,
                IS_rate_all_days,
                correctly_assign_link_rate_all_days,
                incorrect_assign_link_rate_all_days,
                precision_all_days,
                recall_all_days,
                f1_score_all_days
                ]
            
                # Ajouter les noms des lignes
        data_with_row_names = [[row_names[i]] + [round(value, 2) for value in data[i]] for i in range(len(row_names))]

        filename= os.path.join(t2p_save_path,'metrics.csv')
                # Écrire les données dans un fichier CSV
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file, delimiter=';')
            writer.writerow([''] + column_names)  # Écrire les noms des colonnes
            writer.writerows(data_with_row_names)  # Écrire les données avec les noms des lignes
        print(f"CSV file '{filename}' created successfully.")

    if option ==3:
         
        row_names = ['TP_rate', 'TN_rate', 'FP_rate', 'FN_rate', 'IS_rate', 'correctly_assign_link_rate', 'incorrect_assign_link_rate', 'precision', 'recall', 'f1_score']
        column_names = [days[index_of_second_day], days[index_of_third_day]]

                        # Créer une liste de listes pour les données
        data = [
                TP_rate_all_days,
                TN_rate_all_days,
                FP_rate_all_days,
                FN_rate_all_days,
                IS_rate_all_days,
                correctly_assign_link_rate_all_days,
                incorrect_assign_link_rate_all_days,
                precision_all_days,
                recall_all_days,
                f1_score_all_days
                ]
            
                # Ajouter les noms des lignes
        data_with_row_names = [[row_names[i]] + [round(value, 2) for value in data[i]] for i in range(len(row_names))]

        filename= os.path.join(t2p_save_path,'metrics.csv')
        
                # Écrire les données dans un fichier CSV
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file, delimiter=';')
            writer.writerow([''] + column_names)
            writer.writerows(data_with_row_names)  # Écrire les données avec les noms des lignes
        print(f"CSV file '{filename}' created successfully.")

    
         

In [ ]:
def multi_day_evaluation(manual_matches,all_s2p_idxs, option):

    corr_assign_link_all_days = []
    TP_rate_all_days = []
    TN_rate_all_days = []
    FP_rate_all_days = []
    FN_rate_all_days = []
    IS_rate_all_days = []
    incorr_assign_link_all_days = []
    precision_all_days = []
    recall_all_days = []
    f1_score_all_days = []

    if option==1: 
        for day in range(1 , t2p_match_mat.shape[1]): 

            print(f"Day {day}")

            # obtain pairs (P8-P9..P14)
            manual_matches_elements = [(sublist[0], sublist[day]) if sublist is not None and len(sublist) > day else (None, None) for sublist in manual_matches]
            all_s2p_idxs_elements = [(sublist[0], sublist[day]) if sublist is not None and len(sublist) > day else (None, None) for sublist in all_s2p_idxs]

            print("manual_matches_elements:", manual_matches_elements)
            print("all_s2p_idxs_elements:", all_s2p_idxs_elements)


            TP_rate, TN_rate, FP_rate, FN_rate, IS_rate, correctly_assign_link_rate, incorrect_assign_link_rate, precision, recall, f1_score = calculate_metrics_pairs(manual_matches_elements,all_s2p_idxs_elements)

            corr_assign_link_all_days.append(correctly_assign_link_rate)
            TP_rate_all_days.append(TP_rate)
            TN_rate_all_days.append(TN_rate)
            FP_rate_all_days.append(FP_rate)
            FN_rate_all_days.append(FN_rate)
            IS_rate_all_days.append(IS_rate)
            incorr_assign_link_all_days.append(incorrect_assign_link_rate)
            precision_all_days.append(precision)
            recall_all_days.append(recall)
            f1_score_all_days.append(f1_score)
        
        write_metrics_to_csv(TP_rate_all_days, TN_rate_all_days, FP_rate_all_days, FN_rate_all_days, IS_rate_all_days, corr_assign_link_all_days, incorr_assign_link_all_days, precision_all_days, recall_all_days, f1_score_all_days, option)
            
            
    if option==2: 
                manual_matches_elements = [(sublist[0], sublist[index_of_second_day]) if sublist is not None  else (None, None) for sublist in manual_matches]
                # Récupérer l'élément 1 et l'élément 5 de chaque sous-liste dans all_s2p_idxs_days
                all_s2p_idxs_elements = [(sublist[0], sublist[1]) if sublist is not None else (None, None) for sublist in all_s2p_idxs]
                print("manual_matches_elements:", manual_matches_elements)
                print("all_s2p_idxs_elements:", all_s2p_idxs_elements)


                TP_rate, TN_rate, FP_rate, FN_rate, IS_rate, correctly_assign_link_rate, incorrect_assign_link_rate, precision, recall, f1_score = calculate_metrics_pairs(manual_matches_elements,all_s2p_idxs_elements)

                corr_assign_link_all_days.append(correctly_assign_link_rate)
                TP_rate_all_days.append(TP_rate)
                TN_rate_all_days.append(TN_rate)
                FP_rate_all_days.append(FP_rate)
                FN_rate_all_days.append(FN_rate)
                IS_rate_all_days.append(IS_rate)
                incorr_assign_link_all_days.append(incorrect_assign_link_rate)
                precision_all_days.append(precision)
                recall_all_days.append(recall)
                f1_score_all_days.append(f1_score)

                write_metrics_to_csv(TP_rate_all_days, TN_rate_all_days, FP_rate_all_days, FN_rate_all_days, IS_rate_all_days, corr_assign_link_all_days, incorr_assign_link_all_days, precision_all_days, recall_all_days, f1_score_all_days, option)


    if option==3:
                manual_matches_elements = [(sublist[0], sublist[index_of_second_day]) if sublist is not None  else (None, None) for sublist in manual_matches]
                # Récupérer l'élément 1 et l'élément 5 de chaque sous-liste dans all_s2p_idxs_days
                all_s2p_idxs_elements = [(sublist[0], sublist[1]) if sublist is not None else (None, None) for sublist in all_s2p_idxs]

                print("manual_matches_elements:", manual_matches_elements)
                print("all_s2p_idxs_elements:", all_s2p_idxs_elements)


                TP_rate, TN_rate, FP_rate, FN_rate, IS_rate, correctly_assign_link_rate, incorrect_assign_link_rate, precision, recall, f1_score = calculate_metrics_pairs(manual_matches_elements,all_s2p_idxs_elements)

                corr_assign_link_all_days.append(correctly_assign_link_rate)
                TP_rate_all_days.append(TP_rate)
                TN_rate_all_days.append(TN_rate)
                FP_rate_all_days.append(FP_rate)
                FN_rate_all_days.append(FN_rate)
                IS_rate_all_days.append(IS_rate)
                incorr_assign_link_all_days.append(incorrect_assign_link_rate)
                precision_all_days.append(precision)
                recall_all_days.append(recall)
                f1_score_all_days.append(f1_score)
            

                manual_matches_elements = [(sublist[0], sublist[index_of_third_day]) if sublist is not None  else (None, None) for sublist in manual_matches]
                # Récupérer l'élément 1 et l'élément 5 de chaque sous-liste dans all_s2p_idxs_days
                all_s2p_idxs_elements = [(sublist[0], sublist[2]) if sublist is not None else (None, None) for sublist in all_s2p_idxs]


                print("manual_matches_elements:", manual_matches_elements)
                print("all_s2p_idxs_elements:", all_s2p_idxs_elements)
          

                TP_rate, TN_rate, FP_rate, FN_rate, IS_rate, correctly_assign_link_rate, incorrect_assign_link_rate, precision, recall, f1_score = calculate_metrics_pairs(manual_matches_elements,all_s2p_idxs_elements)

                corr_assign_link_all_days.append(correctly_assign_link_rate)
                TP_rate_all_days.append(TP_rate)
                TN_rate_all_days.append(TN_rate)
                FP_rate_all_days.append(FP_rate)
                FN_rate_all_days.append(FN_rate)
                IS_rate_all_days.append(IS_rate)
                incorr_assign_link_all_days.append(incorrect_assign_link_rate)
                precision_all_days.append(precision)
                recall_all_days.append(recall)
                f1_score_all_days.append(f1_score)

                write_metrics_to_csv(TP_rate_all_days, TN_rate_all_days, FP_rate_all_days, FN_rate_all_days, IS_rate_all_days, corr_assign_link_all_days, incorr_assign_link_all_days, precision_all_days, recall_all_days, f1_score_all_days, option)
            

    return {
            'TP_rate': TP_rate_all_days,
            'TN_rate': TN_rate_all_days,
            'FP_rate': FP_rate_all_days,
            'FN_rate': FN_rate_all_days,
            'IS_rate': IS_rate_all_days,
            'correct_links_rate': corr_assign_link_all_days,
            'incorrect_links_rate': incorr_assign_link_all_days,
            'precision': precision_all_days,
            'recall': recall_all_days,
            'f1_score': f1_score_all_days
        }

In [ ]:
result = multi_day_evaluation(manual_matches, all_s2p_idxs, option)
#metrics[path] = result

In [ ]:
print(metrics)

## FIGURES

### Pairwise evaluation

In [ ]:
# Load all metrics.csv tables
def load_metrics_csv(path):
    metrics = pd.read_csv(path, delimiter=';')
    return metrics

#Pairwise evaluation (P8-P9, P9-P10, P10-P11, P11-P12, P12-P13, P13-P14)

mtc_t2p_aff_chan0_all= load_metrics_csv(os.path.join(parent_directory, 't2p_aff_chan0_all_', 'metrics.csv'))
mtc_t2p_aff_chan1_all= load_metrics_csv(os.path.join(parent_directory, 't2p_affine_chan1_all_', 'metrics.csv'))
mtc_t2p_rig_chan1_all= load_metrics_csv(os.path.join(parent_directory, 't2p_rigid_chan1_all_', 'metrics.csv'))
mtc_t2p_P8_9= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_9', 'metrics.csv'))
mtc_t2p_P8_10= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_10', 'metrics.csv'))
mtc_t2p_P8_11= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_11', 'metrics.csv'))
mtc_t2p_P8_12= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_12', 'metrics.csv'))
mtc_t2p_P8_13= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_13', 'metrics.csv'))
mtc_t2p_P8_14= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_14', 'metrics.csv'))

mtc_t2p_P8_P9_P14= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_9_14', 'metrics.csv'))
mtc_t2p_P8_P10_P14= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_10_14', 'metrics.csv'))
mtc_t2p_P8_P11_P14= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_11_14', 'metrics.csv'))
mtc_t2p_P8_P12_P14= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_12_14', 'metrics.csv'))
mtc_t2p_P8_P13_P14= load_metrics_csv(os.path.join(parent_directory, 't2p_P8_13_14', 'metrics.csv'))

print(mtc_t2p_aff_chan0_all)
# Merge the DataFrames on the first column
mtc_t2p_aff_chan1_pairs = mtc_t2p_P8_9
for df in [mtc_t2p_P8_10, mtc_t2p_P8_11, mtc_t2p_P8_12, mtc_t2p_P8_13, mtc_t2p_P8_14]:
    mtc_t2p_aff_chan1_pairs = pd.merge(mtc_t2p_aff_chan1_pairs, df, on=mtc_t2p_aff_chan1_pairs.columns[0], suffixes=('', '_'+df.columns[1]))
print(mtc_t2p_aff_chan1_pairs)

output_csv_path = os.path.join(parent_directory, 'mtc_t2p_aff_chan1_pairs.csv')
mtc_t2p_aff_chan1_pairs.to_csv(output_csv_path, index=False, sep=';')
print(f"DataFrame saved to {output_csv_path}")

# Initialiser le DataFrame avec la première colonne et la colonne P14 du premier DataFrame
mtc_t2_aff_chan1_3days = mtc_t2p_P8_P9_P14[[mtc_t2p_P8_P9_P14.columns[0], 'P14']].rename(columns={'P14': 'P9'})

# Fusionner les DataFrames en utilisant la première colonne comme clé et la colonne P14 pour les valeurs
for df, day in zip([mtc_t2p_P8_P10_P14, mtc_t2p_P8_P11_P14, mtc_t2p_P8_P12_P14, mtc_t2p_P8_P13_P14], ['P10', 'P11', 'P12', 'P13']):
    mtc_t2_aff_chan1_3days = pd.merge(mtc_t2_aff_chan1_3days, df[[df.columns[0], 'P14']].rename(columns={'P14': day}), on=mtc_t2_aff_chan1_3days.columns[0])

print(mtc_t2_aff_chan1_3days)

output_csv_path = os.path.join(parent_directory, 'mtc_t2p_aff_chan1_3days.csv')
mtc_t2_aff_chan1_3days.to_csv(output_csv_path, index=False, sep=';')
print(f"DataFrame saved to {output_csv_path}")

#dfs= [mtc_t2p_aff_chan0_all,mtc_t2p_aff_chan1_all,mtc_t2p_rig_chan1_all,mtc_t2p_aff_chan0_pairs]
dfs_dict = {
    'Aff_tdT_AD': mtc_t2p_aff_chan1_all,
    'Aff_GCamp_AD': mtc_t2p_aff_chan0_all,
    'Rig_tdT_AD': mtc_t2p_rig_chan1_all,
    'Aff_tdT_PW': mtc_t2p_aff_chan1_pairs,
    'Aff_tdT_3days': mtc_t2_aff_chan1_3days
}

['Aff_Int_AD', 'Aff_Act_AD', 'Rig_Int_AD', 'Aff_Int_PW']




In [ ]:
import numpy as np
from matplotlib.colors import to_rgba
cmap_list = ['C0', 'C1', 'C2', 'C3']
from matplotlib.colors import to_rgba
cmap_list_bis = ['C0', 'C1', 'C2', 'C3','C4']

dfs_colors = [to_rgba(cmap) for cmap in cmap_list]

def generate_transparency_gradient(num_colors, idx):
    base_color = dfs_colors
    gradient = []
    for i in range(num_colors):
        alpha = 0.2 + 0.9 * (i / (num_colors - 1)) 
        color_with_alpha = np.append(base_color[:3], alpha)  # Ajouter l'alpha à la couleur de base
        gradient.append(color_with_alpha)
    return gradient

In [ ]:
import numpy as np
from matplotlib.colors import to_rgba
import matplotlib.pyplot as plt

cmap_list = ['C0', 'C1', 'C2', 'C3', 'C4']
dfs_colors = [to_rgba(cmap) for cmap in cmap_list]

def generate_transparency_gradient(num_colors, idx):
    base_color = np.array(dfs_colors[idx])
    gradient = []
    for i in range(num_colors):
        alpha = 0.3 + 0.7 * (i / (num_colors - 1))   
        color_with_alpha = np.append(base_color[:3], alpha)  # Ajouter l'alpha à la couleur de base
        gradient.append(color_with_alpha)
    return gradient

for idx, (data, base_color) in enumerate(zip(dfs_dict, dfs_colors)):
    print(data)
    df = dfs_dict[data]
    print(df)
    f1_pcs_rcl = df[df[df.columns[0]].isin(['precision', 'recall', 'f1_score'])]
    rates = df[df[df.columns[0]].isin(['TP_rate', 'TN_rate', 'FP_rate','FN_rate','IS_rate'])]
    
    # Générer un dégradé de couleurs pour les tracés
    color_gradient = generate_transparency_gradient(len(f1_pcs_rcl), idx=idx)
    
    # Plot f1, precision, recall
    plt.figure(figsize=(3, 3))
    for (index, row), color in zip(f1_pcs_rcl.iterrows(), color_gradient):
        plt.plot(df.columns[1:], row[1:], marker='o', markersize=4, label=row[0], color=color)
        plt.ylim(0, 1.05)
        plt.yticks([0, 0.5, 1])
        plt.xlabel('Age (postnatal days)')
        ax = plt.gca()
        x_ticks = range(len(df.columns) - 1)
        ax.set_xticks(x_ticks)
        x_tick_labels = [''] * len(x_ticks)
        x_tick_labels[0] = df.columns[1]
        x_tick_labels[-1] = df.columns[-1]
        ax.set_xticklabels(x_tick_labels)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  # Placer la légende à droite
    plt.show()
    
    # Générer un dégradé de couleurs pour les tracés
    color_gradient = generate_transparency_gradient(len(rates), idx=idx)
    
    # Plot rates
    plt.figure(figsize=(3, 3))
    for (index, row), color in zip(rates.iterrows(), cmap_list_bis):
        plt.plot(df.columns[1:], row[1:], marker='o', markersize=4, label=row[0], color=color)
        plt.ylim(0, 1.05)
        plt.yticks([0, 0.5, 1])
        plt.xlabel('Age (postnatal days)')
        ax = plt.gca()
        x_ticks = range(len(df.columns) - 1)
        ax.set_xticks(x_ticks)
        x_tick_labels = [''] * len(x_ticks)
        x_tick_labels[0] = df.columns[1]
        x_tick_labels[-1] = df.columns[-1]
        ax.set_xticklabels(x_tick_labels)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  # Placer la légende à droite
    plt.show()

In [ ]:

labels = ['Aff_Int_AD', 'Aff_Act_AD', 'Rig_Int_AD', 'Aff_Int_PW']

plt.figure(figsize=(3, 3))
for color, (data, df) in zip(cmap_list, dfs_dict.items()):
    f1_score_row = df[df[df.columns[0]] == 'f1_score']
    print(data)
    print(f1_score_row)
    for index, row in f1_score_row.iterrows():
        plt.plot(df.columns[1:], row[1:], marker='o', markersize=4, label=f'{data}', color=color)
        

plt.ylim(0, 1.05)
plt.yticks([0, 0.5, 1])
plt.ylabel('F1 Score')

ax = plt.gca()
x_ticks = range(len(df.columns) - 1)
ax.set_xticks(x_ticks)
x_tick_labels = [''] * len(x_ticks)
x_tick_labels[0] = df.columns[1]
x_tick_labels[-1] = df.columns[-1]
ax.set_xticklabels(x_tick_labels)


plt.xlabel('Age (postnatal days)')
plt.legend()
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.show()

### All day evaluation

In [ ]:

mtc_t2p_aff_chan0_all_track= pd.read_csv(os.path.join(parent_directory, 't2p_aff_chan0_all_', 'metrics_tracking_all_days.csv'),  names=['Metric', 'Value'], delimiter=';')
mtc_t2p_aff_chan1_all_track= pd.read_csv(os.path.join(parent_directory, 't2p_affine_chan1_all_', 'metrics_tracking_all_days.csv'),  names=['Metric', 'Value'], delimiter=';')
mtc_t2p_rig_chan1_all_track= pd.read_csv(os.path.join(parent_directory, 't2p_rigid_chan1_all_', 'metrics_tracking_all_days.csv'),  names=['Metric', 'Value'], delimiter=';')

print(mtc_t2p_aff_chan0_all_track)
print(mtc_t2p_aff_chan1_all_track)
print(mtc_t2p_rig_chan1_all_track)



In [ ]:
all_precision = []
all_f1 = []
all_recall = []

for data in [mtc_t2p_aff_chan1_all_track, mtc_t2p_aff_chan0_all_track, mtc_t2p_rig_chan1_all_track]:
    precision = data.loc[data['Metric'] == 'Precision', 'Value'].values[0]
    recall = data.loc[data['Metric'] == 'Recall', 'Value'].values[0]
    f1 = data.loc[data['Metric'] == 'F1', 'Value'].values[0]
    all_precision.append(precision)
    all_recall.append(recall)
    all_f1.append(f1)

print(all_precision)
print(all_recall)
print(all_f1)

labels = ['Aff_tdT_AD', 'Aff_GCamp_AD', 'Rig_tdT_AD']


# Création du barplot pour les valeurs de F1 avec légende
plt.figure(figsize=(3, 3))
for i, (label, f1) in enumerate(zip(labels, all_f1)):
    plt.bar(i, f1, color=['C0', 'C1', 'C2'][i], width=0.5, label=label)
plt.ylabel('F1 Score')
plt.ylim(0, 1.05)
plt.yticks([0, 0.5, 1])
plt.xticks(ticks=range(len(labels)), labels=[''] * len(labels))
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', which='both', bottom=False, top=False)
plt.legend()
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

# Création du barplot pour les valeurs de précision avec légende
plt.figure(figsize=(3, 3))
for i, (label, precision) in enumerate(zip(labels, all_precision)):
    plt.bar(i, precision, color=['C0', 'C1', 'C2'][i], width=0.5, label=label)
plt.ylabel('Precision')
plt.ylim(0, 1.05)
plt.yticks([0, 0.5, 1])
plt.xticks(ticks=range(len(labels)), labels=[''] * len(labels))
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', which='both', bottom=False, top=False)
#plt.legend()
plt.show()

# Création du barplot pour les valeurs de rappel avec légende
plt.figure(figsize=(3, 3))
for i, (label, recall) in enumerate(zip(labels, all_recall)):
    plt.bar(i, recall, color=['C0', 'C1', 'C2'][i], width=0.5, label=label)
plt.ylabel('Recall')
plt.ylim(0, 1.05)
plt.yticks([0, 0.5, 1])
plt.xticks(ticks=range(len(labels)), labels=[''] * len(labels))
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', which='both', bottom=False, top=False)
#plt.legend()
plt.show()

